In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import modules.utils as Utils
from modules.regex_patterns import reviews_paths
from collections import Counter
from underthesea import word_tokenize

### 14. Load dữ liệu `standard_data.csv`

In [3]:
raw_data = pd.read_csv("./data/preprocessing_data.csv")

In [4]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,NaN,lần đầu mua tại ưng ý vải mặc mát mua quần lót...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,NaN,giao hàng nhanh xịn xò đặt hàng xong chủ động ...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,NaN,vừa vặn hàng chất lượng giá rẻ vầy tội không m...,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,NaN,sản phẩm chất lượng đóng gói kĩ nhiệt tình ủng...,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,NaN,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
220023,Nơ xinh đúng chất công chúa đẹpppppp,1,NaN,nơ xinh đúng chất công chúa đẹp,NaN
220024,"Rất dễ thương, đẹp",1,NaN,dễ thương đẹp,dễ_thương
220025,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,NaN,đẹp lắm mọi người ơi học,mọi_người đẹp_lắm_luôn
220026,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,NaN,mua món giao đủ vừa rẻ vừa chất lượng thích ủn...,ủng_hộ chất_lượng giao_đủ


In [5]:
raw_data = raw_data.replace(np.nan, "")
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại ưng ý vải mặc mát mua quần lót...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh xịn xò đặt hàng xong chủ động ...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vừa vặn hàng chất lượng giá rẻ vầy tội không m...,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ nhiệt tình ủng...,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
220023,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,
220024,"Rất dễ thương, đẹp",1,,dễ thương đẹp,dễ_thương
220025,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm mọi người ơi học,mọi_người đẹp_lắm_luôn
220026,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng thích ủn...,ủng_hộ chất_lượng giao_đủ


### 15. Xóa các `boost_words` khỏi `processing_comment`

In [6]:
raw_data['processing_comment'] = raw_data.apply(lambda row: Utils.removeBoostWord(row['processing_comment'], row['boost_words']), axis=1)

In [7]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại vải mặc mát mua quần lót quần ...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,đặt hàng xong chủ động nhắn tin xác nhận gói h...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,hàng vầy tội không mu hả các bác,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,kĩ nhiệt tình lần sau cảm ơn,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,dài dài,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
220023,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,
220024,"Rất dễ thương, đẹp",1,,đẹp,dễ_thương
220025,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm ơi học,mọi_người đẹp_lắm_luôn
220026,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món vừa rẻ vừa thích tiếp,ủng_hộ chất_lượng giao_đủ


### 16, Tokenize bằng `underthesea`

In [8]:
def undertheseaTokenize(pText):
    return " ".join([re.sub(" ", "_", word) for word in word_tokenize(str(pText))])

In [9]:
raw_data['tokenize'] = raw_data['processing_comment'].apply(lambda comment: undertheseaTokenize(comment))

In [10]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words,tokenize
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại vải mặc mát mua quần lót quần ...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...,lần đầu mua tại vải mặc mát mua quần_lót quần_...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,đặt hàng xong chủ động nhắn tin xác nhận gói h...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò,đặt hàng xong chủ_động nhắn_tin xác_nhận gói h...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,hàng vầy tội không mu hả các bác,giá_rẻ chất_lượng vừa_vặn,hàng vầy tội không mu hả các bác
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,kĩ nhiệt tình lần sau cảm ơn,ủng_hộ đóng_gói chất_lượng sản_phẩm,kĩ nhiệt_tình lần sau cảm_ơn
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,dài dài,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...,dài dài
...,...,...,...,...,...,...
220023,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,,nơ xinh đúng chất công_chúa đẹp
220024,"Rất dễ thương, đẹp",1,,đẹp,dễ_thương,đẹp
220025,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm ơi học,mọi_người đẹp_lắm_luôn,đẹp lắm ơi học
220026,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món vừa rẻ vừa thích tiếp,ủng_hộ chất_lượng giao_đủ,mua món vừa rẻ vừa thích tiếp


### 17. Kết hợp 3 cột `tokenize`, `boost_words` và `emoji` lại thành một cột duy nhất là `standard_comment`

In [11]:
raw_data['standard_comment'] = raw_data.apply(lambda row: Utils.genStandardComment(row['tokenize'], row['boost_words'], row['emoji']), axis=1)

In [12]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words,tokenize,standard_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại vải mặc mát mua quần lót quần ...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...,lần đầu mua tại vải mặc mát mua quần_lót quần_...,lần đầu mua tại vải mặc mát mua quần_lót quần_...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,đặt hàng xong chủ động nhắn tin xác nhận gói h...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò,đặt hàng xong chủ_động nhắn_tin xác_nhận gói h...,đặt hàng xong chủ_động nhắn_tin xác_nhận gói h...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,hàng vầy tội không mu hả các bác,giá_rẻ chất_lượng vừa_vặn,hàng vầy tội không mu hả các bác,hàng vầy tội không mu hả các bác giá_rẻ chất_l...
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,kĩ nhiệt tình lần sau cảm ơn,ủng_hộ đóng_gói chất_lượng sản_phẩm,kĩ nhiệt_tình lần sau cảm_ơn,kĩ nhiệt_tình lần sau cảm_ơn ủng_hộ đóng_gói c...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,dài dài,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...,dài dài,dài dài giao_hàng_nhanh ủng_hộ tuyệt_vời chất_...
...,...,...,...,...,...,...,...
220023,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,,nơ xinh đúng chất công_chúa đẹp,nơ xinh đúng chất công_chúa đẹp
220024,"Rất dễ thương, đẹp",1,,đẹp,dễ_thương,đẹp,đẹp dễ_thương
220025,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm ơi học,mọi_người đẹp_lắm_luôn,đẹp lắm ơi học,đẹp lắm ơi học mọi_người đẹp_lắm_luôn
220026,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món vừa rẻ vừa thích tiếp,ủng_hộ chất_lượng giao_đủ,mua món vừa rẻ vừa thích tiếp,mua món vừa rẻ vừa thích tiếp ủng_hộ chất_lượn...


### 18. Xóa các cột thừa đi

In [13]:
raw_data = raw_data.drop(columns=['emoji', 'processing_comment', 'boost_words', 'tokenize'])

In [14]:
raw_data = raw_data.reindex(columns=['raw_comment', 'standard_comment', 'label'])

In [15]:
raw_data

,raw_comment,standard_comment,label
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",lần đầu mua tại vải mặc mát mua quần_lót quần_...,1
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",đặt hàng xong chủ_động nhắn_tin xác_nhận gói h...,1
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",hàng vầy tội không mu hả các bác giá_rẻ chất_l...,1
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,kĩ nhiệt_tình lần sau cảm_ơn ủng_hộ đóng_gói c...,1
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,dài dài giao_hàng_nhanh ủng_hộ tuyệt_vời chất_...,1
...,...,...,...
220023,Nơ xinh đúng chất công chúa đẹpppppp,nơ xinh đúng chất công_chúa đẹp,1
220024,"Rất dễ thương, đẹp",đẹp dễ_thương,1
220025,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,đẹp lắm ơi học mọi_người đẹp_lắm_luôn,1
220026,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,mua món vừa rẻ vừa thích tiếp ủng_hộ chất_lượn...,1


### 19. Ghi ra file

In [ ]:
raw_data.to_csv("./data/standard_data.csv", index=False)